## <font color = teal> Introduction to training models </font>

Training can be performed either with a single yaml file or with several yaml files located in the same directory.

To train a model, you'll need csv files to tell a model which parts of the data (i.e., which ECGs) are used as training data, which part as validation data and which part as testing data, and yaml files based on these csv files for detailed configurations. The csv files can be created by following the introductions in the notebook [Introduction to data handling](1_introduction_data_handling.ipynb). Yaml files can be created with the notebooks [Yaml files of database-wise split for training and testing](2_physionet_DBwise_yaml_files.ipynb) and [Yaml files of stratified split for training and testing](2_physionet_stratified_yaml_files.ipynb).

------


First, check out the yaml files in `/configs/training/` which one you want to use or if you want to make one of your own. The yaml file (in this case, `train_smoke.yaml`) should have the following attributes

```
# INITIAL SETTINGS
train_file: train_split_1_1.csv
val_file: val_split_1_1.csv

# TRAINING SETTINGS
batch_size: 10
num_workers: 0
epochs: 1
lr: 0.003000
weight_decay: 0.000010

# VALIDATION SETTINGS
threshold: 0.5

# DEVICE CONFIGS
device_count: 1

```

where `train_file` refers to a csv file which is used for training phase of epoch(s). It consists of the paths for ECG recordings, patients' gender and age, and labels used in the classification. `val_file` refers to a csv file which is used in the validation phase of epoch(s).

The script for this phase is `train_model.py`. You should first check the path `csv_root` that it points to the right location in the `data` directory (and its subdirectories). The attribute is set to find the csv file of the testing data. Note that as it's concatenated with the `os.path.join` method, you may only need to change the last path component.

<font color ='red'>**NOTE!**</font> The `device_count` attribute in the yaml file should be considered. It refers to the number of GPUs which are used in training. This reflects to the `batch_size` attribute since the batch size should be divided by device count. Obviously, the result of the division should be a positive integer. 

### <font color = teal> CUDA configurations </font>

If you want to spesify the CUDA devices you use in training, you can use the `CUDA_VISIBLE_DEVICES` variable. For example, if you want to run only the first GPU, you can write `CUDA_VISIBLE_DEVICES=0` as a first argument in a command line as follows:

```
CUDA_VISIBLE_DEVICES=0 python script.py
```

As an alternative, you can spesify the CUDA devices as an environment variable in the python scripts as follows:

```
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
```

### <font color = teal> Saving the results </font>

Trained model(s), training history as well as ROC curves and logits of the validation phase will be saved in a subdirectory of the `experiments` directory. ROC curves will have a directory of their own. They are drawn in the first, middle and last epochs; The rest will be saved after the last epoch. Each file and directory will be named after the used yaml file or the directory where the yaml files exist. 

Model will be saved in a `pth` format, ROC curves as `png` images, history as a `pickle` file, and logits as a `csv` file. In the logits csv file, the labels are used as columns and filenames as indexes so that each row contains the validation phase predictions made for the file which is named at the index. 

By default, training history will contain train_loss, micro/macro AUROC scores and micro/macro average precision scores of both training and validation phase as well as Physionet Challenge 2021 scoring metrics (more information about this scoring metric can be found [here under **Scoring**](https://moody-challenge.physionet.org/2021/)). Also, labels, number of epochs, batch size, learning rate, optimizer, criterion and paths for csv files used in training and validation are saved. These can be modified in the `train_utils` script in `./src/modeling/` if wanted. There are a `history` dictionary located in the first lines of the `train` function.

If other metrics are wanted to be computed, you can compute them in the `metrics` scripts in `./src/modeling/`. There can be found the `cal_multilabel_metrics` function, in which the wanted metrics are computed and where you can add new functions. The predicted labels are also one-hot-encoded there, so this format of the predictions is available.

### <font color = teal> Terminal commands </font>

Run a terminal command which consist of the script and the yaml file *or* the directory where all the yaml files are located, so one of the followings

```
python train_model.py train_smoke.yaml
python train_model.py train_stratified_smoke
```

-----------------

## <font color = teal> Example: Smoke testing </font>

<font color = red>**NOTE!**</font> <font color = green> **Here, the assumption is that *the data is preprocessed*. If that's not the case, you should use, for example, the original data directory, such as the** `smoke_data` **directory.** The paths for ECGs will be different in the csv files depending on whether preprocessing has been used or not.</font>

### <font color = teal> One yaml file </font>

Let's use the presented yaml file `train_smoke.yaml` from `/configs/training/` for some smoke testing. The csv files `train_split_1_1.csv` and `val_split_1_1.csv` have already been constructed with the script `create_data_csvs.py`. (More details about this in the notebook [Introduction for Data Handling](1_introduction_data_handling.ipynb)) That said, we are training a model using the stratified split of the data. The first rows from both csvs are as follows

**Train csv**
```
path,age,gender,fs,426783006,426177001,164934002,427084000,164890007,39732003,164889003,59931005,427393009,270492004
./data/physionet_preprocessed_smoke/G12EC/E00008_preprocessed.mat,76.0,Male,500.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/G12EC/E00006_preprocessed.mat,65.0,Male,500.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
./data/physionet_preprocessed_smoke/G12EC/E00005_preprocessed.mat,83.0,Male,500.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/G12EC/E00004_preprocessed.mat,75.0,Male,500.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...
```

**Val csv**
```
path,age,gender,fs,426783006,426177001,164934002,427084000,164890007,39732003,164889003,59931005,427393009,270492004
./data/physionet_preprocessed_smoke/G12EC/E00001_preprocessed.mat,-1.0,Female,500.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/G12EC/E00003_preprocessed.mat,-1.0,Male,500.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/INCART/I0020_preprocessed.mat,59.0,Female,257.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
./data/physionet_preprocessed_smoke/INCART/I0050_preprocessed.mat,70.0,Male,257.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/PTB_PTBXL/HR00008_preprocessed.mat,48.0,Male,500.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/ChapmanShaoxing_Ningbo
...
```

The path in `train_model.py` should be set as below (note the last path component):

```
csv_root = os.path.join(os.getcwd(), 'data', 'split_csvs', 'stratified_smoke')
```

Now you should be ready to perform the training:

```
python train_model.py train_smoke.yaml
```

The trained model, the logits and training history can be found as `train_smoke.pth`, `train_smoke_val_logits.csv` and `train_smoke_history.pickle` in the `experiments` directory. The ROC curve is saved in the `ROC_train_smoke` directory as `roc-e1.png`, named after the number of the epoch on which it's been drawn.

### <font color = teal> Multiple yaml files in a directory </font>

The idea is similar here: Now you should locate all the yaml files in the same directory. The `train_stratified_smoke` directory in `/configs/training/` was created when the yaml files were created with the notebook [Yaml files of Stratified Split for Training and Testing](2_physionet_stratified_yaml_files.ipynb). There are four yaml files named as `split_1_1.yaml`, `split_1_2.yaml`, `split_1_3.yaml` and `split_1_4.yaml` in the directory. Each yaml file has different csv files for training data (`train_file`) and validation data (`val_file`), for example in the two first yaml files, they are set as follows:

`split_1_1.yaml`:
```
# INITIAL SETTINGS
train_file: train_split_1_1.csv
val_file: val_split_1_1.csv

# TRAINING SETTINGS
batch_size: 10
num_workers: 0
epochs: 1
lr: 0.003000
weight_decay: 0.000010

# VALIDATION SETTINGS
threshold: 0.5

# DEVICE CONFIGS
device_count: 1

```

`split_1_2.yaml`:
```
# INITIAL SETTINGS
train_file: train_split_1_2.csv
val_file: val_split_1_2.csv

# TRAINING SETTINGS
batch_size: 10
num_workers: 0
epochs: 1
lr: 0.003000
weight_decay: 0.000010

# VALIDATION SETTINGS
threshold: 0.5

# DEVICE CONFIGS
device_count: 1


```

All the files are constructed from the same stratified split where training data is from the databases G12EC, INCART, PTB_PTBXL, ChapmanShaoxing_Ningbo. 

Again, the path in `train_model.py` should still be set as below

```
csv_root = os.path.join(os.getcwd(), 'data', 'split_csvs', 'stratified_smoke')
```

Terminal command for training is now

```
python train_model.py train_stratified_smoke
```

Trained models are now saved as `split_1_1.pth`, `split_1_2.pth`, `split_1_3.pth` and `split_1_4.pth` in the `train_stratified_smoke` subdirectory (named after the directory in which the yaml files are located) of the `experiments` directory. Similarly, there are four similarly named `pickle` files for each training history, a csv file of logits and ROC curves can be found from four different directories that are also named after the yaml files.